In [2]:
import sys
import os

# Add the parent directory of 'packages' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from packages.workflows.converter.gmail_utils import init_gmail_service

gmail = init_gmail_service()
gmail

/home/duyennt/Demi/MIC/SUPPORT/Test_code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
gmail.get

In [4]:
import os
def normalize_subject(subject: str) -> str:
    """
    Chuẩn hóa subject: loại bỏ khoảng trắng thừa ở đầu và cuối.
    """
    if not subject:
        return ""
    return subject.strip()

def load_allowed_subjects(
    excel_path: str, sheet_name: str = None, keyword: str = "subject"
) -> list:
    """
    Đọc danh sách subject từ file Excel.
    - Tự động tìm cột chứa keyword.
    - Bỏ các cột Unnamed.
    """
    if not os.path.exists(excel_path):
        # logging.error(f"Không tìm thấy file {excel_path}")
        return []

    # 1. Đọc toàn bộ sheet mà không lấy header
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

    # 2. Tìm dòng header thực sự (dòng có keyword)
    header_row_idx = None
    for i, row in df.iterrows():
        # tìm keyword trong row (bỏ qua NaN)
        if any(
            isinstance(cell, str) and keyword.lower() in cell.lower()
            for cell in row
            if cell
        ):
            header_row_idx = i
            break
    if header_row_idx is None:
        # logging.error(
        #     f"Không tìm thấy cột chứa từ khóa '{keyword}' trong file {excel_path}"
        # )
        return []

    # 3. Đọc lại sheet với header đúng
    df_table = pd.read_excel(excel_path, sheet_name=sheet_name, header=header_row_idx)

    # 4. Loại bỏ các cột Unnamed
    df_table = df_table.loc[:, ~df_table.columns.str.contains("^Unnamed")]

    # 5. Tìm cột chứa keyword
    matched_cols = [
        col for col in df_table.columns if keyword.lower() in str(col).lower()
    ]
    if not matched_cols:
        # logging.error(f"Không tìm thấy cột chứa từ khóa '{keyword}' sau khi đọc bảng.")
        return []

    # 6. Lấy dữ liệu từ cột đầu tiên match keyword
    col_name = matched_cols[0]
    subjects = (
        df_table[col_name].dropna().apply(lambda x: normalize_subject(str(x))).tolist()
    )

    # logging.info(f"Đã load {len(subjects)} ALLOWED_SUBJECTS từ {excel_path}")
    return subjects

In [5]:
load_allowed_subjects("/home/duyennt/Demi/MIC/SUPPORT/Test_code/test/data/2025 - Mail Checking Daily.xlsx", sheet_name="Email 2025", keyword='Subject email')

['r5wdGRZk8ux3inrd21GLYWnvu',
 'HỖ TRỢ HỦY LỆNH HOÀN TIỀN OP 03OP2412523026F0F71A',
 'Onepay payment from Merchant',
 'hi - ortal aloni Aloni & stein BSD,llc <asbsdllc@gmail.com>',
 'RE: QR BIC (OP_BIC10) Thông báo chậm tạm ứng doanh số 27/12-29/12/2024 - PVS716097',
 'EKKO – GD PayOut pending EKFRICOUT27D60822BF, EKVUACOUTADD9583A0A, EKCARCOUTCDD86D727D, EKACICOUT6263EF82BB',
 'Request For Account Information',
 'Hieu ldol',
 'Error During Transaction',
 'Lỗi trong quá trình giao dịch',
 'Nạp tiền điện thoại nhưng không vào tài khoản của số đt',
 'Đã mua thẻ garena mà ko được',
 'Michel Salmeron <michelsalmeron21@gmail.com>',
 'already fill finish Vietnam evisa 3month single entry,show fee$25,how to pay this evisa fee.',
 'GIAO DỊCH HOÀN TRẢ 188,312 đồng 16/12/2024 KHÁCH CHƯA NHẬN ĐƯỢC TIỀN (dh: 6771239 + 6771240)',
 'Mí Chừ Sùng <sungmichudmx10@gmail.com>',
 'ZARA- TRA SOÁT- 78900890',
 'OnePay gui file doi soat 20241228_TECHCOMBANK_INSTALLMENT.csv',
 'OnePay gui file doi soat 202412

NameError: name '__file__' is not defined